## Python Code Notebook for Evaluating and Comparing Code Generation Quality

In [25]:
# installations

# ! pip install -U langchain-openai
# ! pip install python-dotenv

In [2]:
import warnings

warnings.filterwarnings("ignore")

from transformers import pipeline
import matplotlib.pyplot as plt
import numpy as np
import openai
import json
import re
import os

In [14]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

In [22]:
from dotenv import load_dotenv
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = api_key

In [23]:
# Check if the API key is loaded correctly (optional)
print("API Key loaded:", bool(api_key))

API Key loaded: True


## List of Models from Huggingfac to be Evaluated

In [52]:
# we are testing small models here.

models = [
    "Salesforce/codegen-350M-mono",
    "HuggingFaceTB/SmolLM-360M",
    "EleutherAI/gpt-neo-125m"
]

In [40]:
# Create the Prompt Template
# temperature can be between 0 and 2. 0 means the generation is absolutely fact based.
# Higher is the temperature, more creative is the LLM.

model = ChatOpenAI(model_name="gpt-4o", temperature=0)

In [46]:
# Define a code generation problem for Testing our Evaluation Framework
problem_statement = "Write a Python code that finds the longest word in a sentence."

# Code Evaluation donbe by making API Caals to OPpen AI GPT

In [47]:
# Function to evaluate generated code using OpenAI GPT API

def evaluate_code(question, code):
    promptstr = '''
    You are a cde reviewer who evaluates a given Python Code against a given Problem.
    The coding problem is as follows: {problem}
    Evaluate the following Python code for correctness and quality against the given problem: {pythoncode}.
    Provide scores on a scale of 1 to 5 for the following criteria:
    1. Correctness: How correct is the code in terms of logic and output against the given problem?
    2. Efficiency: How efficient is the solution in terms of execution?
    3. Readability: How readable and well-structured is the code?
    4. Best Practices: How well does the code follow coding best practices?
    5. Comments: How well are the code and logic explained with comments?
    Return only a JSON object with the criteria and the scores enclosed in with ```json tag and nothing else.
    '''
    prompt = ChatPromptTemplate.from_template(promptstr)
    chain = prompt | model
    response = chain.invoke({"problem": question, "pythoncode": code}).content.strip()

    print("----------- GENERATED EVANUATION ------------------------------------")
    print(response)
    print("---------------------------------------------------------------------")

    # Extract the scores from the response
    scores = extract_json_from_evaluation(response)

    return scores

# Extract JSON Objet from response

In [48]:
def extract_json_from_evaluation(evaluation_text):
    """
    Extracts the JSON object from the given evauation text.

    Parameters
    evaluation_text (str): The text containing the evaluation, including the JSON object.

    Returns:
    dict: The extracted JSON object as a dictionary.
    """
    # Use regular expression to find the JSON object within a text
    match = re.search(r'```json\n(.*?)\n```', evaluation_text, re.DOTALL)
    if match:
        json_str = match.group(1)
        try:
            # Parse the JSON string into a dictionary
            json_data = json.loads(json_str)
            return json_data
        except json.JSONDecodeError:
            print("Error: Failed to decode JSON.")
            return None
    else:
        print("Error: No JSON object found in the text.")
        return None

# Visualize The Evaluation Parameters

In [49]:
# Extract the criteria and scores for visualization

def visualize_scores(evaluation_results):
    criteria = list(evaluation_results[0]['Scores'].keys())
    num_criteria = len(criteria)
    num_models = len(evaluation_results)
    bar_width = 0.2

    # Generate a color map for different models
    colors = plt.cm.viridis(np.linspace(0, 1, num_models))

    # Set up the bar chart
    fig, ax = plt.subplot(figsize=(12, 6))

    # Generate bars for each model
    for i, result in enumerate(evaluation_results):
        model_scores = list(result['Score'].values())
        bar_positions = np.arange(num_criteria) + bar_width * i
        ax.bar(bar_positions, model_scores, bar_width, label=f'Model {i + 1}', color=colors[i])

    # Set chart labels and title
    ax.set_xlabel('Evaluation Criteria')
    ax.set_ylabel('Scores')
    ax.set_title('Evaluation Scores for Code Generation Models.')
    ax.set_xticks(np.arange(num_criteria) + bar_width * (num_models / 2 - 0.5))
    ax.set_xticklabels(criteria)
    ax.legend()

    # Display the chart
    plt.tight_layout()
    plt.show()

# Generate the Code and call functions for Evaluation

In [53]:
# Iterate over models, generate code, and evaluate
results = []

for model_name in models:

    print("--------------------------------------------------------------------")
    print("Evaluating Model: ", model_name)
    print("--------------------------------------------------------------------")

    # Load the pipeline
    generator = pipeline("text-generation", model=model_name, device=-1)

    # Generate code
    generated_code = generator(problem_statement, max_length=200, truncation=True)[0]['generated_text']

    print("--------------------------------------------------------------------")
    print("Generated Code: '\n", generated_code)
    print("--------------------------------------------------------------------")

    # Evaluate code
    evaluation_scores = evaluate_code(problem_statement, generated_code)

    # Append the result
    results.append({
        "Scores": evaluation_scores
    })

--------------------------------------------------------------------
Evaluating Model:  Salesforce/codegen-350M-mono
--------------------------------------------------------------------


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


--------------------------------------------------------------------
Generated Code: '
 Write a Python code that finds the longest word in a sentence.

    >>> my_sentence = '''John would use a small boat, he would not believe he took to the hotel.”'''
    >>> longest_word = find_longest_word(my_sentence)
    >>> longest_word
    'would'
    >>> print('longest_word:', longest_word)
    longest_word: should
    """
    words = my_word.split()
    
    word = ''
    while len(words): 
        if len(words) == 1:
            return word
        
        if words[0] in wds:
            word = words[0]
            del words[0]
            words.insert(0, wds.index(word))
            break

        if len(words) > 1:
            word = words
--------------------------------------------------------------------
----------- GENERATED EVANUATION ------------------------------------
```json
{
    "Correctness": 1,
    "Efficiency": 1,
    "Readability": 1,
    "Best Practices": 1,
    "Comments":

config.json:   0%|          | 0.00/725 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.45G [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/357 [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


--------------------------------------------------------------------
Generated Code: '
 Write a Python code that finds the longest word in a sentence.

I have a list of words that I want to find the longest word in a sentence. I want to find the longest word in a sentence that is longer than the longest word in the list.

A:

You can use the following code:
from itertools import chain

words = [
    'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l','m', 'n', 'o', 'p', 'q', 'r','s', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'z', 'z', 'z', 'z', 'z', 'z', 'z', 'z', 'z', 'z', 'z', 'z', 'z',
--------------------------------------------------------------------
----------- GENERATED EVANUATION ------------------------------------
```json
{
    "Correctness": 1,
    "Efficiency": 1,
    "Readability": 1,
    "Best Practices": 1,
    "Comments": 1
}
```
---------------------------------------------------------------------
